$\underline{\text{Modèle de Verhulst, partie 3 : introduction au chaos et fractales de Liapounov}}$

Bienvenue dans cette troisième et dernière partie du modèle de Verhulst ! Je te conseille de d'abord commencer par la vidéo ou le fichier PDF associés à cette partie, avant de te lancer dans les joies de la simulation !

Cet outil va te permettre d'exécuter du code Python facilement, à travers ce que j'ai déjà codé, ou de petits exercices où tu dois toi-même coder !

L'objectif est que tu découvres la puissance de l'informatique pour ce genre de calculs et pour l'intuition qu'elle peut t'apporter, mais aussi ses limites. Bonne découverte !

PS : Au début, appuie sur Cell->Run All pour tout lancer. Pour activer une cellule de code particulière (comme après une modification), clique dessus et appuie sur Cells->Run Cells, ou Shift+Enter ;)

In [1]:
import os
import sys
import random as rd
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import ColumnDataSource, output_file, show

output_notebook(hide_banner=True)

$\underline{\text{Tests sur la suite pour }0 \leq \mu \leq 4}$

In [2]:
#fonction f_µ
def f_mu(mu):
    return lambda x:mu*x*(1-x)

def calcul_suivant(mu,x, fig):
    f=f_mu(mu)
    fig.x(f(x),0,color="green")
    fig.line([x,x,f(x)],[x,f(x),f(x)])
    fig.x(f(x),0,color="green")
    return f(x)

def affichage_evolution(mu,x0,nb_etapes):
    print("On note :")
    print("- x0 la valeur initiale de la suite")
    print("- nb_etapes le nombre d'itérations de la suite")
    plt.close()
    fig = figure(width=800, height=400, title="Evolution de la suite")
    liste=np.array(np.linspace(0,1,num=40000))
    fig.line(liste, f_mu(mu)(liste), color='blue')
    fig.line([0,1],[0,1], color="black")
    x=x0
    fig.x(x,0,color="green")
    for i in range(nb_etapes):
        x=calcul_suivant(mu, x, fig)
    show(fig)

In [3]:
interact(affichage_evolution, mu=widgets.FloatSlider(min=0, max=4, step=0.05, value=2, continuous_update=False), x0=widgets.FloatSlider(min=0, max=1, step=0.05, value=0.1, continuous_update=False), nb_etapes=widgets.IntSlider(min=0, max=200, step=5, value=40, continuous_update=False))

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='mu', max=4.0, step=0.05), F…

<function __main__.affichage_evolution(mu, x0, nb_etapes)>

Tu peux donc découvrir les itérés de la suite pour toutes les valeurs de $\mu$ entre 0 et 4, et voir que le comportement n'est plus aussi simple que précédemment dans certains cas. On va donc s'intéresser principalement dans la suite à ce qu'il se passe en $\mu =4$, avec l'apparition du $\textbf{chaos}$.

$\underline{\text{Pour }\mu=4\text{ : un chaos compréhensible}}$

On se place ici à $\mu=4$ et on trace les 9 dernières itérations de la suite, avec leur méthode de détermination graphique. Les points verts correspondent à toutes les valeurs prises jusqu'alors par la suite, et elles semblent se répartir, pour la plupart, dans tout le segment [0,1]. 

Cependant, pour $x_0=0,25$ ou $x_0=0,5$, on observe que seul un nombre restreint de valeurs sont prises, faisant parti des antécédents des points fixes répulsifs de la suite. Mais en obtenant précisément cette valeur, la suite reste stationnaire.

In [4]:
liste_couleur=[(255,0,0),(255,32,0),(255,64,0),(255,96,0),(255,128,0),(255,160,0),(255,192,0),(255,224,0),(255,255,0)]

f_4=f_mu(4)

def calcul_suivant2(mu,x, fig):
    f=f_mu(mu)
    fig.x(f(x),0,color="green")
    return f(x)

def affichage_etape(mu,x, fig, etape):
    f=f_mu(mu)
    fig.line([x,x,f(x)],[x,f(x),f(x)], color=liste_couleur[etape])
    fig.x(f(x),0,color="green")
    return f(x)

def affichage_evolution(x0,nb_etapes):
    print("On note :")
    print("- x0 la valeur initiale de la suite")
    print("- nb_etapes le nombre d'itérations de la suite")
    fig = figure(width=800, height=400, title="Evolution de la suite")
    liste=np.array(np.linspace(0,1,num=40000))
    fig.line(liste, f_4(liste), color='blue')
    fig.line([0,1],[0,1], color="black")
    x=x0
    fig.x(x,0,color="green")
    for i in range(nb_etapes-8):
        x=calcul_suivant2(4, x, fig)
    for i in range(nb_etapes-8,nb_etapes):
        x=affichage_etape(4, x, fig, nb_etapes-i)
    show(fig)

In [5]:
interact(affichage_evolution, x0=widgets.FloatSlider(min=0, max=1, step=0.05, value=0.1, continuous_update=False), nb_etapes=widgets.IntSlider(min=0, max=200, step=5, value=40, continuous_update=False))

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='x0', max=1.0, step=0.05), I…

<function __main__.affichage_evolution(x0, nb_etapes)>

$\underline{Sensibilité~aux~Conditions~Initiales}$

On s'intéresse à la sensibilité au Conditions Initiales (CI) pour la fonction $f_4$. Celle-ci est vérifiée car la dynamique associée à $f_4$ est chaotique. 

Nous allons donc considérer l'évolution des suites A, B, C et D dont les termes initiaux sont très proches : $B_0=A_0+10^{-4}$, $C_0=A_0+10^{-7}$ et $D_0=A_0+10^{-10}$. Malgré cette proximité, toutes ces suites finissent par avoir des comportements très différents : la vitesse de "séparation" des suites varie en fonction de la valeur initiale de A, notée a. 

Deux graphiques permettent d'observer cette séparation : le tracé des valeurs succesives pour les 4 suites, et l'évolution de B, C et D par rapport à A au cours des itérations.

In [6]:
#Sensibilité aux valeurs initiales
def sensibilite_CI(a, n_max):
    print("On note :")
    print("- a la valeur initiale de la suite")
    print("- n_max le nombre maximal d'itérations")
    liste_a=[a]
    b=a+0.0001
    liste_b=[b]
    c=a+0.00000001
    liste_c=[c]
    d=a+0.00000000001
    liste_d=[d]
    liste_etape=[0]
    for i in range(1,n_max):
        liste_etape.append(i)
        liste_a.append(f_4(liste_a[-1]))
        liste_b.append(f_4(liste_b[-1]))
        liste_c.append(f_4(liste_c[-1]))
        liste_d.append(f_4(liste_d[-1]))

    fig = figure(width=470, height=300, title="Sensibilité aux CI : évolution des suites de premiers termes proches")
    fig.line(liste_etape, liste_a, legend_label="A", color='black')
    fig.line(liste_etape, liste_b, legend_label="B", color='blue')
    fig.line(liste_etape, liste_c, legend_label="C", color='red')
    fig.line(liste_etape, liste_d, legend_label="D", color='green')
    fig.legend.location = "top_right"       

    liste_b_diff=abs(np.array(liste_b)-np.array(liste_a))
    liste_c_diff=abs(np.array(liste_c)-np.array(liste_a))
    liste_d_diff=abs(np.array(liste_d)-np.array(liste_a))
    fig1 = figure(width=470, height=300, title="Sensibilité aux CI : évolution des écarts à A")
    fig1.line(liste_etape, liste_b_diff, legend_label="|B-A|", color='blue')
    fig1.line(liste_etape, liste_c_diff, legend_label="|C-A|", color='red')
    fig1.line(liste_etape, liste_d_diff, legend_label="|D-A|", color='green')
    fig1.legend.location = "top_left"       
    show(row(fig, fig1))

In [7]:
interact(sensibilite_CI, a=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1, continuous_update=False), n_max=widgets.IntSlider(min=0, max=200, step=20, value=40, continuous_update=False))

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='a', max=1.0), IntSlider(val…

<function __main__.sensibilite_CI(a, n_max)>

Pour en avoir une vision plus classique, tu peux observer ici l'évolution de deux pendules doubles, en plaçant leurs masses à des positions originales très proches.

In [8]:
def pendule_double(m1, m2, l1, l2, O1, O2, v1, v2, nb_etapes, T_max):
    theta1=[O1]
    theta2=[O2]
    vit1=[v1]
    vit2=[v2]
    T=T_max/nb_etapes
    for t in range(nb_etapes):
        theta1.append((theta1[-1]+T*vit1[-1])%(2*np.pi))
        theta2.append((theta2[-1]+T*vit2[-1])%(2*np.pi))
        vit1.append(vit1[-1]+T*(-m2*l1*np.sin(theta1[-2]-theta2[-2])*np.cos(theta1[-2]-theta2[-2])*vit1[-1]**2-m2*l2*np.sin(theta1[-2]-theta2[-2])*vit2[-1]**2-m1*9.81*np.sin(theta1[-2])-m2*9.81*np.sin(theta1[-2]-theta2[-2])*np.cos(theta2[-2]))/(m1*l1+m2*l1*(np.sin(theta1[-2]-theta2[-2]))**2))
        #print(np.sin(theta1[-2]-theta2[-2]), theta1[-2]-theta2[-2])
        vit2.append(vit2[-1]+T*((m1+m2)*l1*np.sin(theta1[-2]-theta2[-2])*vit1[-2]**2+m2*l2*np.sin(theta1[-2]-theta2[-2])*np.cos(theta1[-2]-theta2[-2])*vit2[-1]**2+(m1+m2)*9.81*np.sin(theta1[-2]-theta2[-2])*np.cos(theta1[-2]))/(m1*l2+m2*l2*(np.sin(theta1[-2]-theta2[-2]))**2))
    return theta1, theta2

def pendules_proches(m1, m2, l1, l2, O1, O2, v1, v2, eO1, eO2, ev1, ev2, nb_etapes, T_max):
    theta11, theta12 = pendule_double(m1, m2, l1, l2, O1, O2, v1, v2, nb_etapes, T_max)
    theta21, theta22 = pendule_double(m1, m2, l1, l2, O1+eO1, O2+eO2, v1+ev1, v2+ev2, nb_etapes, T_max)
    coord1=np.zeros((len(theta11), 2, 2), dtype=float)
    coord2=np.zeros((len(theta21), 2, 2), dtype=float)
    for i in range(len(theta11)):
        coord1[i,0,0]=l1*np.sin(theta11[i])
        coord1[i,0,1]=-l1*np.cos(theta11[i])
        coord1[i,1,0]=l1*np.sin(theta11[i])+l2*np.sin(theta12[i])
        coord1[i,1,1]=-l1*np.cos(theta11[i])-l2*np.cos(theta12[i])
        coord2[i,0,0]=l1*np.sin(theta21[i])
        coord2[i,0,1]=-l1*np.cos(theta21[i])
        coord2[i,1,0]=l1*np.sin(theta21[i])+l2*np.sin(theta22[i])
        coord2[i,1,1]=-l1*np.cos(theta21[i])-l2*np.cos(theta22[i])
    #print(coord1)
    return coord1, coord2


In [18]:
print("m : masse, l : longueur, O : angle initial par rapport à la verticale basse, v: vitesse initiale, eO : écart en angle \ninitial, ev : écart en vitesse initial, nb_etapes : nombre de discrétisations, T_max : temps d'évolution")
fonction_coord_pendule_double=interactive(pendules_proches, m1=widgets.FloatSlider(min=0, max=2, step=0.2, value=1, continuous_update=False), m2=widgets.FloatSlider(min=0, max=2, step=0.2, value=1, continuous_update=False), l1=widgets.FloatSlider(min=0, max=1, step=0.2, value=0.4, continuous_update=False), l2=widgets.FloatSlider(min=0, max=1, step=0.2, value=0.4, continuous_update=False), O1=widgets.FloatSlider(min=0, max=2*np.pi, step=np.pi/8, value=0, continuous_update=False), O2=widgets.FloatSlider(min=0, max=2*np.pi, step=np.pi/8, value=0, continuous_update=False), v1=widgets.FloatSlider(min=0, max=1, step=0.1, value=0, continuous_update=False), v2=widgets.FloatSlider(min=0, max=1, step=0.1, value=0, continuous_update=False), eO1=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.01, continuous_update=False), eO2=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.01, continuous_update=False), ev1=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.0, continuous_update=False), ev2=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.0, continuous_update=False), nb_etapes=widgets.IntSlider(min=10, max=1000, step=10, value=50, continuous_update=False), T_max=widgets.FloatSlider(min=1, max=20, step=0.5, value=5, continuous_update=False))
display(fonction_coord_pendule_double)

m : masse, l : longueur, O : angle initial par rapport à la verticale basse, v: vitesse initiale, eO : écart en angle 
initial, ev : écart en vitesse initial, nb_etapes : nombre de discrétisations, T_max : temps d'évolution


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='m1', max=2.0, step=0.2), Fl…

$\color{red}{\text{Après chaque changement de paramètre, il faut revalider les deux cellules suivantes (avec Shift+Enter)}}$

In [23]:
coord1, coord2= fonction_coord_pendule_double.result

def affichage_pendule_double(t):
    fig = figure(width=470, height=300, x_range=[-2,2], y_range=[-2,2], title="Sensibilité aux CI : évolution des suites de premiers termes proches")
    fig.line([0,coord1[t,0,0], coord1[t,1,0]], [0,coord1[t,0,1],coord1[t,1,1]], color='black')
    fig.line([0,coord2[t,0,0], coord2[t,1,0]], [0,coord2[t,0,1],coord2[t,1,1]], color='blue')
    show(fig)

In [24]:
interact(affichage_pendule_double, t=widgets.IntSlider(min=0, max=1000, step=1, value=0, continuous_update=True))

interactive(children=(IntSlider(value=0, description='t', max=1000), Output()), _dom_classes=('widget-interact…

<function __main__.affichage_pendule_double(t)>

$\underline{\text{Sensibilité et approximations numériques, un bien mauvais mélange...}}$

Cette sensibilité aux conditions initiales rend très problématique le calcul itératif des termes de la suite logistique pour $\mu=4$, à cause d'un autre phénomène : l'espace de stockage d'un ordinateur est fini, et la plupart des nombres ne peuvent être stockés exactement par l'ordinateur, ce qui le conduit à en prendre des approximations.

Or, ce phénomène d'approximation, couplé à la sensibilité, mène à des comportements numériques différents des résultats exacts.

Pour illustrer cette problématique, considérons : $u_0=\sin^2(\frac{\pi}{384})$.

Or $384=2^7\times 3$ donc $u_7=\sin^2(2^7\frac{\pi}{384})=\sin^2(\frac{\pi}{3})=\frac{3}{4}$.

Or, $\frac{3}{4}$ est un point fixe de $f_4$, donc la suite $(u_n)_{n\in \mathbb{N}}$ est stationnaire à partir du rang 7. Mais ce n'est pas le comportement que nous allons observer numériquement...

In [12]:
def pb_num_sensibilite(nb_etape):
    print("On note :")
    print("- nb_etape le nombre d'itérations de la suite considéré, avec valeur initiale de u0")
    liste_valeurs=[np.sin(np.pi/384)**2]
    for i in range(nb_etape):
        liste_valeurs.append(f_4(liste_valeurs[-1]))
    fig = figure(width=470, height=300, title="Evolution de la suite, plus ou moins stationnaire")
    fig.line([0,nb_etape], [3/4,3/4], legend_label="Limite attendue", color='black')
    fig.line(list(range(nb_etape+1)), liste_valeurs, legend_label="Evolution de la suite", color='blue')
    fig.legend.location = "bottom_right" 
    show(fig)

In [13]:
interact(pb_num_sensibilite, nb_etape=widgets.IntSlider(min=1, max=100, step=1, value=50, continuous_update=False))

interactive(children=(IntSlider(value=50, continuous_update=False, description='nb_etape', min=1), Output()), …

<function __main__.pb_num_sensibilite(nb_etape)>

Ainsi, nous observons qu'à partir d'une certaine valeur, les erreurs de calcul et la sensibilité mènent une suite, stationnaire en théorie, à diverger !

**Après avoir pu observer et simuler par toi-même les concepts introduits dans la première vidéo, n'hésite pas à visionner la deuxième, qui te permettra de découvrir le nombre de Champernowne (qui justifie la dynamique chaotique de $f_4$) et d'avoir une idée de ce qu'il se passe lorsque $\mu\in]\mu_{\infty},4[$.**